In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

from dtale import show

# Load data

In [2]:
fp_pred = pd.read_csv('fp_pred.csv')
prices = pd.read_csv('output_csv/historic_dfs_data_v2.csv')

In [3]:
fp_pred.head()

,Unnamed: 0,player_id,game_id,game_date,reb,reb_pred,player_name,team,pts,pts_pred,...,blk,blk_pred,to,to_pred,fp_fanduel,fp_fanduel_pred,fp_yahoo,fp_yahoo_pred,fp_draftkings,fp_draftkings_pred
0,0,202324,22000743,2021-04-02,5.0,5.894968,Derrick Favors,UTA,4.0,4.727058,...,2.0,1.742050,0.0,0.462477,19.0,19.323977,19.0,19.323977,16.25,17.626077
1,1,1629671,22000743,2021-04-02,0.0,0.508486,Miye Oni,UTA,0.0,1.523207,...,0.0,0.115687,0.0,0.045665,0.0,3.469601,0.0,3.469601,0.00,3.247910
2,2,1627777,22000743,2021-04-02,3.0,1.491486,Georges Niang,UTA,3.0,4.696698,...,0.0,0.199984,1.0,0.426469,7.1,9.058756,7.1,9.058756,7.75,8.787013
3,3,203903,22000743,2021-04-02,2.0,3.424841,Jordan Clarkson,UTA,19.0,11.713884,...,0.0,0.013563,2.0,1.714314,23.9,18.226125,23.9,18.226125,24.00,18.822521
4,4,201144,22000743,2021-04-02,2.0,3.172887,Mike Conley,UTA,13.0,15.386163,...,1.0,0.366885,1.0,2.160271,27.9,28.870261,27.9,28.870261,26.50,28.659534


In [4]:
prices.head()

,date,player_name,team,opp.,score,yh_position,yh_points,yh_salary,fd_position,fd_points,fd_salary,dk_position,dk_points,dk_salary
0,2021-07-01,Giannis Antetokounmpo,mil,v atl,123-112,NaN,NaN,NaN,NaN,NaN,NaN,PF/C,0.00,11200.0
1,2021-07-01,Cam Reddish,atl,@ mil,112-123,NaN,NaN,NaN,NaN,NaN,NaN,SG/SF,20.00,3700.0
2,2021-07-01,Donte DiVincenzo,mil,v atl,123-112,NaN,NaN,NaN,NaN,NaN,NaN,PG,0.00,3000.0
3,2021-07-01,Jordan Nwora,mil,v atl,123-112,NaN,NaN,NaN,NaN,NaN,NaN,SF/PF,0.00,3000.0
4,2021-07-01,Bryn Forbes,mil,v atl,123-112,NaN,NaN,NaN,NaN,NaN,NaN,PG/SG,4.75,3300.0


In [5]:
fp_pred['player_name']

0         Derrick Favors
1               Miye Oni
2          Georges Niang
3        Jordan Clarkson
4            Mike Conley
              ...       
13438     Justin Jackson
13439        Sam Merrill
13440       Jordan Nwora
13441       Axel Toupane
13442      Mikal Bridges
Name: player_name, Length: 13443, dtype: object

In [6]:
def get_best_match(name, name_list, threshold=80):
    best_match, best_score = process.extractOne(name, name_list, scorer=fuzz.token_set_ratio)
    if best_score >= threshold:
        return best_match
    else:
        return None

In [11]:
name_mapping = {}
prices_names = set(prices['player_name'].tolist())


In [16]:
for name in set(fp_pred['player_name']):
    if name == 'Juan Hernangomez':
        continue
    best_match = get_best_match(name, prices_names)
    if best_match:
        name_mapping[name] = best_match
    if best_match and name != best_match:
        print(name, best_match)

Kelly Oubre Jr. Kelly Oubre
Ish Smith Ishmael Smith
Robert Williams III Robert Williams
Lonnie Walker IV Lonnie Walker
James Ennis III James Ennis
Derrick Jones Jr. Derrick Jones
T.J. Leaf TJ Leaf
DeAndre' Bembry DeAndre Bembry
CJ McCollum C.J. McCollum
Wesley Matthews Wes Matthews
Kevin Porter Jr. Kevin Porter
Cam Reynolds Cameron Reynolds
Juancho Hernangomez Juan Hernangomez
Harry Giles III Harry Giles
Timothe Luwawu-Cabarrot Timothe Luwawu
Otto Porter Jr. Otto Porter
JJ Redick J.J. Redick
Svi Mykhailiuk Sviatoslav Mykhailiuk
Michael Porter Jr. Michael Porter
KJ Martin Jeremiah Martin
Gary Payton II Gary Payton
Lou Williams Louis Williams
Reggie Bullock Jr. Reggie Bullock
Danuel House Jr. Danuel House
Willy Hernangomez Juan Hernangomez
Nic Claxton Nicolas Claxton
Larry Nance Jr. Larry Nance
Marvin Bagley III Marvin Bagley
Didi Louzada Didi Louzada Silva
Jaren Jackson Jr. Jaren Jackson
Marcus Morris Sr. Marcus Morris
Charlie Brown Jr. Charlie Brown
Kevin Knox II Kevin Knox


In [26]:
fp_pred['player_name'] = fp_pred['player_name'].map(name_mapping).fillna(fp_pred['player_name'])
fp_pred['team'] = fp_pred['team'].apply(lambda x: x.lower())
# Merge DataFrames on 'player_name' and 'game_date' (date in df2)
merged_df = pd.merge(fp_pred, prices, left_on=['player_name', 'game_date'], right_on=['player_name', 'date'], how='left')

,Unnamed: 0,player_id,game_id,game_date,reb,reb_pred,player_name,team,pts,pts_pred,...,blk,blk_pred,to,to_pred,fp_fanduel,fp_fanduel_pred,fp_yahoo,fp_yahoo_pred,fp_draftkings,fp_draftkings_pred
0,0,202324,22000743,2021-04-02,5.0,5.894968,Derrick Favors,uta,4.0,4.727058,...,2.0,1.742050,0.0,0.462477,19.0,19.323977,19.0,19.323977,16.25,17.626077
1,1,1629671,22000743,2021-04-02,0.0,0.508486,Miye Oni,uta,0.0,1.523207,...,0.0,0.115687,0.0,0.045665,0.0,3.469601,0.0,3.469601,0.00,3.247910
2,2,1627777,22000743,2021-04-02,3.0,1.491486,Georges Niang,uta,3.0,4.696698,...,0.0,0.199984,1.0,0.426469,7.1,9.058756,7.1,9.058756,7.75,8.787013
3,3,203903,22000743,2021-04-02,2.0,3.424841,Jordan Clarkson,uta,19.0,11.713884,...,0.0,0.013563,2.0,1.714314,23.9,18.226125,23.9,18.226125,24.00,18.822521
4,4,201144,22000743,2021-04-02,2.0,3.172887,Mike Conley,uta,13.0,15.386163,...,1.0,0.366885,1.0,2.160271,27.9,28.870261,27.9,28.870261,26.50,28.659534


In [18]:
merged_df

,Unnamed: 0,player_id,game_id,game_date,reb,reb_pred,player_name,team_x,pts,pts_pred,...,score,yh_position,yh_points,yh_salary,fd_position,fd_points,fd_salary,dk_position,dk_points,dk_salary
0,0,202324,22000743,2021-04-02,5.0,5.894968,Derrick Favors,UTA,4.0,4.727058,...,113-106,C,19.0,10.0,C,19.0,4000.0,C,16.25,3200.0
1,1,1629671,22000743,2021-04-02,0.0,0.508486,Miye Oni,UTA,0.0,1.523207,...,113-106,SF,0.0,10.0,SG,0.0,3500.0,SF,0.00,3000.0
2,2,1627777,22000743,2021-04-02,3.0,1.491486,Georges Niang,UTA,3.0,4.696698,...,113-106,PF,7.1,10.0,PF,7.1,3500.0,PF,8.25,3100.0
3,3,203903,22000743,2021-04-02,2.0,3.424841,Jordan Clarkson,UTA,19.0,11.713884,...,113-106,SG,23.9,22.0,PG,23.9,5300.0,PG/SG,25.50,6100.0
4,4,201144,22000743,2021-04-02,2.0,3.172887,Mike Conley,UTA,13.0,15.386163,...,113-106,PG,27.9,25.0,PG,27.9,6400.0,PG,28.00,6600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13438,13438,1628382,42000406,2021-07-20,0.0,0.142233,Justin Jackson,MIL,0.0,0.231057,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13439,13439,1630241,42000406,2021-07-20,0.0,0.374224,Sam Merrill,MIL,0.0,0.423289,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13440,13440,1629670,42000406,2021-07-20,0.0,-0.086099,Jordan Nwora,MIL,0.0,0.643771,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13441,13441,1626253,42000406,2021-07-20,0.0,0.142233,Axel Toupane,MIL,0.0,0.213954,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
import numpy as np

merged_df[merged_df.index.isin(np.random.randint(0, len(merged_df), 10))]

,Unnamed: 0,player_id,game_id,game_date,reb,reb_pred,player_name,pts,pts_pred,ast,...,score,yh_position,yh_points,yh_salary,fd_position,fd_points,fd_salary,dk_position,dk_points,dk_salary
216,216,203926,22000740,2021-04-02,2.0,3.383019,Doug McDermott,13.0,12.310982,3.0,...,97-114,SF,19.9,10.0,SF,19.9,3700.0,SF/PF,21.50,3800.0
2044,2044,1629008,22000806,2021-04-11,11.0,8.935031,Michael Porter Jr.,22.0,21.165085,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3302,3302,201567,22000841,2021-04-15,6.0,7.425484,Kevin Love,5.0,14.557187,2.0,...,101-119,PF,17.2,21.0,PF,17.2,6300.0,PF/C,17.50,6200.0
4102,4102,201152,22000278,2021-04-12,1.0,7.028052,Thaddeus Young,20.0,12.752648,1.0,...,90-101,PF,21.7,20.0,PF,21.7,6000.0,PF/C,22.25,5900.0
5731,5731,201599,22000884,2021-04-21,12.0,2.226687,DeAndre Jordan,7.0,3.339460,2.0,...,103-114,C,25.4,12.0,C,25.4,4600.0,C,26.00,4000.0
6043,6043,1629680,22000949,2021-04-30,1.0,2.706204,Matisse Thybulle,10.0,4.706179,5.0,...,126-104,NaN,NaN,NaN,SF,30.7,3500.0,NaN,NaN,NaN
7043,7043,1627783,22000921,2021-04-26,5.0,7.136386,Pascal Siakam,25.0,25.034200,4.0,...,112-96,PF,37.0,37.0,PF,37.0,9000.0,PF,38.75,8900.0
8861,8861,101150,22001000,2021-05-06,0.0,2.015520,Lou Williams,7.0,7.726086,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9163,9163,1628970,22001064,2021-05-15,5.0,6.808865,Miles Bridges,30.0,16.594774,4.0,...,109-118,PF,45.0,27.0,SF,45.0,6900.0,SF/PF,46.75,7200.0
10092,10092,1630165,22001033,2021-05-11,6.0,4.286047,Killian Hayes,13.0,8.725999,7.0,...,100-119,PG,27.7,13.0,PG,27.7,5200.0,PG,30.00,5400.0


In [19]:
def get_player_info(player_name, game_date):
    prices_f = prices[(prices['player_name'] == player_name) & (prices['date'] == game_date)]
    pred_f = fp_pred[(fp_pred['player_name'] == player_name) & (fp_pred['game_date'] == game_date)]
    
    return prices_f, pred_f

In [24]:
d_prices, d_pred = get_player_info('Michael Porter Jr.', '2021-04-11')

In [29]:
prices[(prices['player_name'].str.contains('Michael Porter'))]# & (prices['date'] == game_date)]

,date,player_name,team,opp.,score,yh_position,yh_points,yh_salary,fd_position,fd_points,fd_salary,dk_position,dk_points,dk_salary
641,2021-06-13,Michael Porter,den,v pho,118-125,SF,36.8,27.0,SF,36.8,6600.0,SF/PF,34.50,6300.0
710,2021-06-11,Michael Porter,den,v pho,102-116,SF,25.5,28.0,SF,25.5,7200.0,SF/PF,27.25,6900.0
905,2021-06-07,Michael Porter,den,@ pho,105-122,SF,26.4,28.0,SF,26.4,7800.0,SF/PF,28.25,7100.0
1027,2021-06-03,Michael Porter,den,@ por,126-115,SF,40.0,28.0,SF,40.0,7600.0,SF/PF,42.75,6900.0
1268,2021-06-01,Michael Porter,den,v por,147-140,SF,47.9,28.0,SF,47.9,6500.0,SF/PF,51.00,7000.0
1526,2021-05-29,Michael Porter,den,@ por,95-115,SF,7.0,33.0,SF,7.0,7200.0,SF/PF,8.75,7200.0
1791,2021-05-27,Michael Porter,den,@ por,120-115,SF,22.5,34.0,PF,22.5,7500.0,SF/PF,24.75,7100.0
2079,2021-05-24,Michael Porter,den,v por,128-109,SF,29.3,34.0,SF,29.3,7600.0,SF/PF,29.50,7000.0
2351,2021-05-22,Michael Porter,den,v por,109-123,NaN,NaN,NaN,SF,39.8,7500.0,SF/PF,39.75,8200.0
2810,2021-05-16,Michael Porter,den,@ por,116-132,SF,11.6,34.0,SF,11.6,7500.0,SF/PF,13.75,8400.0


In [26]:
d_pred

,Unnamed: 0,player_id,game_id,game_date,reb,reb_pred,player_name,pts,pts_pred,ast,...,blk,blk_pred,to,to_pred,fp_fanduel,fp_fanduel_pred,fp_yahoo,fp_yahoo_pred,fp_draftkings,fp_draftkings_pred
2044,2044,1629008,22000806,2021-04-11,11.0,8.935031,Michael Porter Jr.,22.0,21.165085,0.0,...,1.0,0.906374,0.0,1.131726,41.2,35.967524,41.2,35.967524,41.25,35.881765
